In [8]:
# setup for earth engine
import ee
import geemap.core as geemap
import h5py

ee.Authenticate()

ee.Initialize(project='ee-akerr1')

from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive

Mounted at /content/drive
Urban_Heat


# Metadata Information
 **Time**: filename should give year and day of year, the array should have 96 15-minute time-slices starting at 0:00 of that day. I think Chris gave it in UTC order, otherwise its local day. (easy to see if any of those slices has same-time of day temperatures or east west variation).

 **Geolocation**. practice is regular 0.25-degree grid for this, probably with North West corner at (60 N, 180 W).

 **Units**. Some version of degree Kelvin, not sure if scaling was necessary, and what missing data value is. (This product won't have temperatures below zero).

 **Microwave channel**: Ka-band (37 GHz, vertical polarized). If we did the scaling correctly than this should indeed not matter.

In [35]:
filename_base = r"/content/drive/MyDrive/Urban_Heat/mw_lst_2022/MW_LST_DTC_20230101_x1y.h5"

f = h5py.File(filename_base,'r')
dataset = f['TB37V_LST_DTC'] # 96 time slices, 1440 lon values, 600 lat values

dataset[95][0]

# def list_dataset(name, node):
#   if isinstance (node, h5py.Dataset):
#     print(name)

# def ls_dataset(name,node):
#     if isinstance(node, h5py.Dataset):
#         print(node)



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

A note on dataset shape: We have 600 lat indexes, which means we have 150 degrees of data. From Thomas: probably starts with North West corner (60N, 180W), but didn't fully confirm.

In [ ]:
lst_data = f['TB37V_LST_DTC']
lst_shape = lst_data.shape

# wavelengths
